# 2. gyakorlat

## Célok:
* LP modell felépítése adott problémából *(folytatás)*
* LP megoldása pulp programcsomag használatával
* Mátrixjáték kevert egyensúlya LP modelljének megismerése

### Modellezési feladat, pulp

*1. feladat*
<br>Egy számítógépes játékban minél ütőképesebb hadsereget szeretnénk létrehozni. Tizenkét óra játékidő áll rendelkezésünkre és 24 dollár van a számlánkon. A hadsereg katonákból és páncélos katonákból áll. Egy katona kiképzése egy órányi időt és három dollárnyi befektetést igényel. Egy páncél legyártása két órányi időt és két dollárnyi befektetést igényel. Egy sima katona harcértéke egy egység, egy páncélos katona harcértéke négy egység.
<br>a) Írjunk fel egy LP modellt a problémára!  **- múlt héten szerepelt**

*Megoldás:*
<br> $x_1$: a kiképzendő katonák száma (eltekintünk az egészértékű elvárástól)
<br> $x_2$: a legyártandó páncélok száma (eltekintünk az egészértékű elvárástól)

$$
\begin{array}{rll}
\max & x_1 + 3 x_2 & \\
\text{f.h.} & x_1 + 2x_2 & \leq 12 \\
& 3x_1 + 2x_2 & \leq 24 \\
& x_1 - x_2 & \geq 0 \\
& x_1, x_2 & \geq 0
\end{array}
$$

b) Írjuk fel a lineáris programozási modellt a pulp programcsomag segítségével, oldjuk meg, és olvassuk le a megoldást!

In [ ]:
# Ha a pulp még nem került telepítésre:
pip install pulp
# Általában külön cellában való futtatást igényel

# Ezután egészítsük ki az alábbi hiányos kódot:

In [1]:
# Megoldás:

# programcsomag betöltése:
from pulp import *

# LpProblem változó létrehozása a model tárolására:
model = LpProblem('Hadsereg', LpMaximize)

x1 = LpVariable('katona', lowBound=0 , cat = 'Continuous')
x2 = LpVariable('páncél', lowBound=0 , cat = 'Continuous')
# lowBound: opcionális alsó korlát
# cat: változó jellege (folytonos, egészértékű, bináris)

# célfüggvény:
model += x1 + 3*x2 # célfüggvény, majd korlátok

# korlátok:
model += x1 + 2*x2 <= 12, "idő"
model += 3*x1 + 2*x2 <= 24, "pénz"
model += x1 - x2 >= 0, "páncél"

print(model)

# LP megoldása:
megoldas = model.solve()

print(megoldas)
print(LpStatus[megoldas])

# megoldás leolvasása:
if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Célfüggvény érték: z* = ', value(model.objective))
    print('Megoldás: katona* = ', value(x1), ', páncél* = ', value(x2))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

Hadsereg:
MAXIMIZE
1*katona + 3*páncél + 0
SUBJECT TO
idő: katona + 2 páncél <= 12

pénz: 3 katona + 2 páncél <= 24

páncél: katona - páncél >= 0

VARIABLES
katona Continuous
páncél Continuous

1
Optimal
Optimális megoldást találtunk.
Célfüggvény érték: z* =  16.0
Megoldás: katona* =  4.0 , páncél* =  4.0


In [2]:
# Alternativ megoldás:

# from pulp import *

model2 = LpProblem('Hadsereg_alt', LpMaximize)

# tömb változó a változók egyenkénti deklarációja helyett
x = LpVariable.dicts('x',range(2), lowBound=0, cat="Continuous")

# célfüggvény és korlátok ennek megfelelően módosulnak
model2 += x[0] + 3*x[1]

model2 += x[0] + 2*x[1] <= 12, "idő"
model2 += 3*x[0] + 2*x[1] <= 24, "pénz"
model2 += x[0] - x[1] >= 0, "páncél"

print(model2)

megoldas = model2.solve()

print(megoldas)
print(LpStatus[megoldas])

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Célfüggvény érték: z* = ', value(model2.objective))
    print('Megoldás: katona* = ', value(x[0]), ', páncél* = ', value(x[1]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

Hadsereg_alt:
MAXIMIZE
1*x_0 + 3*x_1 + 0
SUBJECT TO
idő: x_0 + 2 x_1 <= 12

pénz: 3 x_0 + 2 x_1 <= 24

páncél: x_0 - x_1 >= 0

VARIABLES
x_0 Continuous
x_1 Continuous

1
Optimal
Optimális megoldást találtunk.
Célfüggvény érték: z* =  16.0
Megoldás: katona* =  4.0 , páncél* =  4.0


c) Van több optimális megoldás?
<br>Ha igen, akkor adjuk meg az összes optimális megoldást!
<br>Ha nem, akkor tegyük most fel, hogy a páncélos katona harcértéke csökken. Meddig kéne legalább csökkennie ahhoz, hogy több optimális megoldás legyen?

*Megoldás:*
<br>Nincs több optimális megoldás.
<br>Mindez változatlanul így marad egész addig, amíg a célfüggvény (eredetileg -1/3) meredeksége szigorúan kisebb marad, mint 1, illetve szigorúan nagyobb marad, mint -1/2.
<br>$x_2$ célfüggvény együtthatója csökkenésével a határ, amibe a meredekség beleütközik a -1/2, amikor $x_2$ célfüggvény egyutthatója 2-re csökken.
<br>A páncélos katona harcértéke 1-gyel nagyobb, mint $x_2$ célfüggvény együtthatója, tehát 3-ig kéne, hogy csökkenjen ahhoz, hogy több optimális megoldás is legyen.

d) Tegyük fel, hogy szeretnénk legalább 8 db páncélt legyártani: $x_2 \geq 8$ új korlát.

In [5]:
from pulp import *

model = LpProblem('Hadsereg', LpMaximize)

x1 = LpVariable('katona', lowBound=0 , cat = 'Continuous')
x2 = LpVariable('páncél', lowBound=0 , cat = 'Continuous')

model += x1 + 3*x2

model += x1 + 2*x2 <= 12, "idő"
model += 3*x1 + 2*x2 <= 24, "pénz"
model += x1 - x2 >= 0, "páncél"
model += x2 >= 8, "túlzás"

print(model)

megoldas = model.solve()

print(megoldas)
print(LpStatus[megoldas])

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Célfüggvény érték: z* = ', value(model.objective))
    print('Megoldás: katona* = ', value(x1), ', páncél* = ', value(x2))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

-1
Infeasible
Az LP-nek nincs megengedett megoldása.


In [7]:
print('Célfüggvény érték: z* = ', value(model.objective))
print('Megoldás: katona* = ', value(x1), ', páncél* = ', value(x2))

Célfüggvény érték: z* =  16.0
Megoldás: katona* =  4.0 , páncél* =  4.0


*2. feladat*

Egy cég kétféle szarvasmarhatápot állít elő, mindkettő csak búzából és lóheréből áll. Az 1-es tápnak legalább 80% búzát kell tartalmaznia, míg a 2-es tápnak legalább 60% lóherét kell tartalmaznia.
<br>Az 1-es táp eladási ára kg-ként 1.50\\$, a 2-es táp eladási ára kg-ként 1.30\\$.
<br>A cég legfeljebb 1000 kg búzát vásárolhat, kg-ját 50 centért, és legfeljebb 800 kg lóherét vásárolhat, kg-ját 40 centért.
<br>a) Fogalmazzon meg egy lineáris programozási modellt a cég profitjának maximalizálására!

*Megoldás:*

Döntési változók:
<br>$x_1$: 1-es táp mennyisége
<br>$x_2$: 2-es táp mennyisége
<br>$y_{11}$: búza 1-es táphoz
<br>$y_{21}$: lóhere 1-es táphoz
<br>$y_{12}$: búza 2-es táphoz
<br>$y_{22}$: lóhere 2-es táphoz
				
Célfüggvény:
$z = 1.5x_1 + 1.3x_2 - 0.5 (y_{11} + y_{12}) - 0.4 (y_{21} + y_{22}) \to \max!$

Korlátok:
* Mennyiségek:
    <br>$x_1 = y_{11} + y_{21}$ $\;\;\;\;\;\;$ ($x_1 \leq y_{11} + y_{21}$ is lehetne)
    <br>$x_2 = y_{12} + y_{22}$ $\;\;\;\;\;\;$ ($x_2 \leq y_{12} + y_{22}$ is lehetne)
	<br>$y_{11} + y_{12} \leq 1000$
	<br>$y_{21} + y_{22} \leq 800$
    <br>
* Arányok:
    <br>$\frac{y_{11}}{y_{11} + y_{21}} \geq 0.8$
    <br>$\frac{y_{22}}{y_{12} + y_{22}} \geq 0.6$
    
    Nemlineáris! Helyette:
    <br>$y_{11} \geq 0.8(y_{11} + y_{21})$
    <br>$y_{22} \geq 0.6(y_{12} + y_{22})$
    <br>
* Nemnegativitás:
    <br>$x_1, x_2, y_{11}, y_{12}, y_{21}, y_{22} \geq 0$

b) A cég elhatározta, hogy mennyiségi kedvezményt ad. Ha több mint 300 kg 1-es tápot ad el, akkor minden egyes 300 kg feletti kg tápot 1.25\\$-ért ad.
<br>Ehhez hasonlóan minden 300 kg feletti kg 2-es tápot 1.00\\$-ért kínál.
<br>Módosítsa a linearáis programot úgy, hogy figyelembe veszi a mennyiségi árengedményt!

*Megoldás:*

A 3. gyakorlaton visszatérünk rá

### Gyakorló feladat: modellezési, grafikus megoldás, pulp

*3. feladat*

A Sörvinus Kft kézműves sört gyárt. Alapanyagként árpamalátát és komlót használ.
<br>Az előállított sör keserűségét és testességét egy skálán mérik:
* Egy adag árpamaláta hozzáadása 0,12 ponttal, egy adag komló 0,15 ponttal növeli a sör keserűségét.
* A sör testességét a komló 0,25 tel növeli, az árpamaláta viszont 0,1 gyel csökkenti

Egy adag árpamaláta 600 Ft-ba, egy adag komló pedig 760 Ft-ba kerül. Milyen arányban keverje az összetevőket a Sörvinus ha a lehető legolcsóbb olyan sört szeretné főzni, amely megfelel az alábbi feltételeknek:
* A sör keserűsége legfeljebb 0,42 lehet.
* A sör testessége 0,35 és 0,46 közé kell, hogy essen.
* A hozzáadott komló mennyisége nem haladhatja meg az árpamaláta mennyiségének kétszeresét.

a) Írjunk fel egy LP modellt a problémára!

*Megoldás:*
<br> $x_1$: árpamaláta (adag)
<br> $x_2$: komló (adag)

$$
\begin{array}{rll}
\min & 600x_1 + 760x_2 & \\
\text{f.h. } & 0.12x_1 + 0.15x_2 & \leq 0.42 \\
& -0.1x_1 + 0.25x_2 & \geq 0.35 \\
& -0.1x_1 + 0.25x_2 & \leq 0.46 \\
& -2x_1 + x_2 & \leq 0 \\
& x_1, x_2 & \geq 0
\end{array}
$$

b) Adjunk meg egy optimális megoldást, és a hozzá tartozó optimális célfüggvény értéket! Az optimális megoldásban mely feltételek inaktívak? Az inaktív feltételek jobb oldalai legfeljebb mennyit változhatnak úgy, hogy a kiválasztott optimális megoldás továbbra is optimális maradjon?

*Megoldás:*
<br>Optimális megoldás: (7/8; 7/4)
<br>Optimális célfüggvény érték: 1855
<br>Inaktív feltételek: első és harmadik (illetve az előjelkorlátok)
<br>Megengedett változás az első korlát jobb oldalán: [0,0525; inf) (avagy a jobb oldal a [0,3675, inf) intervallumon belül)
<br>Megengedett változás a harmadik korlát jobb oldalán: [-0,11; inf) (avagy a jobb oldal a [0,35, inf) intervallumon belül)

c) Írjuk fel a lineáris programozási modellt a pulp programcsomag segítségével, oldjuk meg, és olvassuk le a megoldást!

In [3]:
# Megoldás:

# from pulp import *

model = LpProblem('Sörvinus', LpMinimize)

x1 = LpVariable('maláta', lowBound=0 , cat = 'Continuous')
x2 = LpVariable('komló', lowBound=0 , cat = 'Continuous')

model += 600*x1 + 760*x2

model += 0.12*x1 + 0.15*x2 <= 0.42, "keserűség"
model += -0.1*x1 + 0.25*x2 >= 0.35, "testesség_min"
model += -0.1*x1 + 0.25*x2 <= 0.46, "testesség_max"
model += -2*x1 + x2 <= 0, "komló_maláta_arány"

print(model)

megoldas = model.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Célfüggvény érték: z* = ', value(model.objective))
    print('Megoldás: maláta* = ', value(x1), ', komló* = ', value(x2))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

Sörvinus:
MINIMIZE
760*komló + 600*maláta + 0
SUBJECT TO
keserűség: 0.15 komló + 0.12 maláta <= 0.42

testesség_min: 0.25 komló - 0.1 maláta >= 0.35

testesség_max: 0.25 komló - 0.1 maláta <= 0.46

komló_maláta_arány: komló - 2 maláta <= 0

VARIABLES
komló Continuous
maláta Continuous

Optimális megoldást találtunk.
Célfüggvény érték: z* =  1855.0
Megoldás: maláta* =  0.875 , komló* =  1.75


### Mátrixjáték

<ins>Emlékeztető:</ins>
* Kevert stratégia **x**: ha egy játékosnak $k$ választási lehetősége van
    * ezekhez rendel $x_i, i=1, \dots, k$ valószínűségeket, melyekre teljesül
    * $x_i \in [0,1]$ minden $i=1, \dots, k$-ra
    * $\sum_{i=1}^k x_i = 1$.

A kevert stratégiák jelentősége:
* Sor játékos célja: minél nagyobb garantált minimális kifizetés.
    * Tényleges kifizetés az Oszlop játékos döntésétől is függ!
* Oszlop játékos célja: minél kisebb garantált maximális veszteség.
* **Minden mtx.játékban létezik Sor és Oszlop kevert stratégiái, melyekre az előbbiek egyszerre teljesülnek**

Az így megkapott kevert stratégiapárt a mx.játék <ins>Nash-egyensúly</ins>ának nevezzük.
* Nash-egyensúlyban egyik játékosnak sem éri meg egyoldalúan eltérnie a választott stratégiájától
* egyoldalúan: a másik játékos kevert stratégiájának változatlansága mellett
* nem éri meg: nem növekszik a várható kifizetése
* a Nash-egyensúly egyben a <ins>játék nyeregpontja</ins> (*kifizetési függvény egy nyeregpontját határozza meg*)

<ins>LP modell</ins> a Nash-egyensúlyi kevert stratégiapár meghatározására:

Legyen a mtx.játék kifzetési mátrixa az alábbi:
$$
A =\left[ \begin{array}{ccc}
 10 & 8 & 7 \\
 \phantom{1}4 & 9 & 6 \\
\end{array} \right]
$$

Sor kevert stratégiája: $x_1, x_2$ valószínűségek
* Oszlop elsőt választ: Sor várható kifizetése $10x_1 + 4x_2$
* Oszlop második: $8x_1 + 9x_2$
* Oszlop harmadik: $7x_1 + 6x_2$

Bármilyen stratégiát is választ Oszlop, Sor maximalizálja, amit el tud érni
* maximalizálja a várható kifizetései közül a legkisebbet (azok minimumát)
* azaz: maximalizál egy $v$ értéket, amely alsó korlátja mindegyiknek
$$
\color{blue}{
\begin{array}{r}
10x_1 + 4x_2 \geq v \\
8x_1 + 9x_2 \geq v \\
7x_1 + 6x_2 \geq v
\end{array}
}
$$

Célfüggvény: $\hspace{8mm}\color{blue}{\max v}$
<br> Valószínűségek: $\;\color{blue}{x_1 + x_2 = 1}$
<br> $\hspace{29mm}\color{blue}{x_1, x_2 \geq 0}$

* Vegyük észre, hogy az újonnan bevezetett $v$ változóra nincs előjelkikötés!
* $v$ az az érték, amekkora nyereséget Sor mindenképp tud garantálni magának
* Az optimális megoldásban felvett $v^*$ értéket nevezzük a **játék értékének**
* Ha $v^* >0$: a játék *Sornak kedvez*; $v^* <0$: a játék *Oszlopnak kedvez*; $v^* =0$: a játék *igazságos*

In [4]:
# Sor LP megoldásal pulppal

# from pulp import *

sor = LpProblem('sor', LpMaximize)

x1 = LpVariable('x1', lowBound=0, cat='Continuous')
x2 = LpVariable('x2', lowBound=0, cat='Continuous')

# Az alábbi sorral van lehetőségem egyszerre több változót hozzáadni a modellhez:
# x = LpVariable.dicts('x',range(2), lowBound=0, cat="continuous")

v = LpVariable('v', cat='Continuous')
# Vegyük észre az előjelkikötés (nemnegativitás) hiányát!

sor += v

sor += 10*x1 + 4*x2 - v >= 0, "oszlop stratégia 1"
sor += 8*x1 + 9*x2 - v >= 0, "oszlop stratégia 2"
sor += 7*x1 + 6*x2 - v >= 0, "oszlop stratégia 3"
sor += x1 + x2 == 1, "valószínűségi változók"

print(sor)

megoldas = sor.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: v* =', value(sor.objective))
    print('Sor kevert stratégiája: x1 =', value(x1), ', x2 =', value(x2))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

sor:
MAXIMIZE
1*v + 0
SUBJECT TO
oszlop_stratégia_1: - v + 10 x1 + 4 x2 >= 0

oszlop_stratégia_2: - v + 8 x1 + 9 x2 >= 0

oszlop_stratégia_3: - v + 7 x1 + 6 x2 >= 0

valószínűségi_változók: x1 + x2 = 1

VARIABLES
v free Continuous
x1 Continuous
x2 Continuous

Optimális megoldást találtunk.
Játék értéke: v* = 7.0
Sor kevert stratégiája: x1 = 1.0 , x2 = 0.0


Ugyanez az Oszlop szempontjából: kevert stratégiája $y_1, y_2, y_3$ valószínűségek
* Sor elsőt választ: Oszlop várható kifizetése $10y_1 + 8y_2 + 7y_3$
* Sor második: $4y_1 + 9y_2 + 6y_3$

Bármilyen stratégiát is választ Sor, Oszlop minimalizálja, amit veszíthet
* minimalizálja a várható veszteségei közül a legnagyobbat (azok maximumát)
* azaz: minimalizál egy $w$ értéket, amely felső korlátja mindegyiknek 
$\color{blue}{
\hspace{20mm}10y_1 + 8y_2 + 7y_3 \leq w \\
\hspace{22mm}4y_1 + 9y_2 + 6y_3 \leq w
}$

Célfüggvény: $\hspace{6mm}\color{blue}{\min w}$
<br> Valószínűségek: $\;\color{blue}{y_1 + y_2 + y_3= 1,}$
<br> $\hspace{29mm} \color{blue}{y_1, y_2, y_3 \geq 0}$

* Vegyük észre, hogy az újonnan bevezetett $w$ változóra szintén nincs előjelkikötés!
* Oszlop garantálni tudja, hogy $w$ értékénél nagyobb veszteséget nem szenved el
* Az optimális megoldásban felvett $w^*$ érték szintén a **játék értéke**
* Azaz ha $w^* >0$: a játék *Sornak kedvez*; $w^* <0$: a játék *Oszlopnak kedvez*; $w^* =0$: a játék *igazságos*

In [5]:
# Oszlop LP megoldásal pulppal

# from pulp import *

oszlop = LpProblem('oszlop', LpMinimize)

# Egyszerre több, azonos karakterisztkájú változót is deklarálhatunk
y = LpVariable.dicts('y',range(3), lowBound=0, cat="Continuous")
w = LpVariable('w', cat='Continuous')
# Vegyük észre az előjelkikötés (nemnegativitás) hiányát!

oszlop += w

# A változólistánk elemenkénti használata (az indexálás 0-tól indul!)
oszlop += 10*y[0] + 8*y[1] + 7*y[2] - w <= 0, "sor stratégia 1"
oszlop += 4*y[0] + 9*y[1] + 6*y[2] - w <= 0, "sor stratégia 2"

# Változólista lehetőséget ad az elemein történő iteráláson
# Az így kapott lista elemeit a pulp lpSum összegző függvényével használhatjuk
oszlop += pulp.lpSum([y[i] for i in range(3)]) == 1, "valószínűségi változók"

print(oszlop)

megoldas = oszlop.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: w* =', value(oszlop.objective))
    print('Oszlop kevert stratégiája:')
    for i in range(3):
        print('y' + str(i+1) + ' =', value(y[i]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

oszlop:
MINIMIZE
1*w + 0
SUBJECT TO
sor_stratégia_1: - w + 10 y_0 + 8 y_1 + 7 y_2 <= 0

sor_stratégia_2: - w + 4 y_0 + 9 y_1 + 6 y_2 <= 0

valószínűségi_változók: y_0 + y_1 + y_2 = 1

VARIABLES
w free Continuous
y_0 Continuous
y_1 Continuous
y_2 Continuous

Optimális megoldást találtunk.
Játék értéke: w* = 7.0
Oszlop kevert stratégiája:
y1 = 0.0
y2 = 0.0
y3 = 1.0


Meg lehetett volna találni az optimális kevert stratégiát LP használata nélkül is?

Sor egyik (tiszta) stratégiája sem dominálja a másikat. Oszlop második stratégiája dominálja a harmadikat. A dominált stratégia kiszűrése után fennmaradó kifizetési mátrix
$$
\left[ \begin{array}{cc}
 10 & 7 \\
 \phantom{1}4 & 6 \\
\end{array} \right]
$$

Ekkor már Sor első stratégiája dominálja a másodikat, így a második sort is törölhetjük a mátrixból. A fennmaradó két kimenetből pedig Oszlop egyértelműen tud választani (7 veszteség 10 helyett).

### Mátrixjáték gyakorló feladatok

*4. feladat*
<br>Az első játékosnak van 2 kavicsa, a második játékosnak 3. Mindkét játékos betesz valahány kavicsot a kezébe (anélkül, hogy a másik játékosnak megmutatná) és utána mindketten kinyitják a kezüket. Ha a kavicsok összege páros, akkor a kavicsok összegének megfelelő pénzt fizet a második az első játékosnak; ha páratlan, akkor viszont a második játékosnak fizet az első a kavicsok összegét.
<br>a) Írjuk fel a játék kifizetési mátrixát!

*Megoldás:*
$$
\left[ \begin{array}{cccc}
 \phantom{-}0 & -1 & \phantom{-}2 & -3 \\
 -1 & \phantom{-}2 & -3 & \phantom{-}4 \\
 \phantom{-}2 & -3 & \phantom{-}4 & -5 \\
\end{array} \right]
$$

b) Határozzuk meg a mátrixjáték egy Nash-egyensúlyi kevert stratégiapárját egy LP segítségével! Ha választhat, melyik szerepben játszana inkább? Mennyit lenne hajlandó áldozni arra, hogy játszhasson?

In [ ]:
# Megoldás:

# from pulp import *

sor = LpProblem('sor', LpMaximize)

x = LpVariable.dicts('x', range(3), lowBound=0, cat='Continuous')
v = LpVariable('v', cat='Continuous')

sor += v

sor += -x[1] + 2*x[2] - v >= 0, "oszlop stratégia 1"
sor += -x[0] + 2*x[1] - 3*x[2] - v >= 0, "oszlop stratégia 2"
sor += 2*x[0] - 3*x[1] + 4*x[2] - v >= 0, "oszlop stratégia 3"
sor += -3*x[0] + 4*x[1] - 5*x[2] - v >= 0, "oszlop stratégia 4"
sor += pulp.lpSum([x[i] for i in range(3)]) == 1, "valószínűségi változók"

print(sor)

megoldas = sor.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: v* =', value(sor.objective))
    print('Sor kevert stratégiája:')
    for i in range(3):
        print('x' + str(i+1) + ' =', value(x[i]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')    
else:
    print('Az LP megoldása valamely problémába ütközött.')

sor:
MAXIMIZE
1*v + 0
SUBJECT TO
oszlop_stratégia_1: - v - x_1 + 2 x_2 >= 0

oszlop_stratégia_2: - v - x_0 + 2 x_1 - 3 x_2 >= 0

oszlop_stratégia_3: - v + 2 x_0 - 3 x_1 + 4 x_2 >= 0

oszlop_stratégia_4: - v - 3 x_0 + 4 x_1 - 5 x_2 >= 0

valószínűségi_változók: x_0 + x_1 + x_2 = 1

VARIABLES
v free Continuous
x_0 Continuous
x_1 Continuous
x_2 Continuous

Optimális megoldást találtunk.
Játék értéke: v* = 0.0
Sor kevert stratégiája:
x1 = 0.25
x2 = 0.5
x3 = 0.25


In [7]:
#from pulp import *

oszlop = LpProblem('oszlop', LpMinimize)

y = LpVariable.dicts('y', range(4), lowBound=0, cat='Continuous')
w = LpVariable('w', cat='Continuous')

oszlop += w

oszlop += -y[1] + 2*y[2] - 3*y[3] - w <= 0, "sor stratégia 1"
oszlop += -y[0] + 2*y[1] - 3*y[2] + 4*y[3] - w <= 0, "sor stratégia 2"
oszlop += 2*y[0] - 3*y[1] + 4*y[2] - 5*y[3] - w <= 0, "sor stratégia 3"
oszlop += pulp.lpSum([y[i] for i in range(4)]) == 1, "valószínűségi változók"

print(oszlop)

megoldas = oszlop.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: w* =', value(oszlop.objective))
    print('Oszlop kevert stratégiája:')
    for i in range(4):
        print('y' + str(i+1) + ' =', value(y[i]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

oszlop:
MINIMIZE
1*w + 0
SUBJECT TO
sor_stratégia_1: - w - y_1 + 2 y_2 - 3 y_3 <= 0

sor_stratégia_2: - w - y_0 + 2 y_1 - 3 y_2 + 4 y_3 <= 0

sor_stratégia_3: - w + 2 y_0 - 3 y_1 + 4 y_2 - 5 y_3 <= 0

valószínűségi_változók: y_0 + y_1 + y_2 + y_3 = 1

VARIABLES
w free Continuous
y_0 Continuous
y_1 Continuous
y_2 Continuous
y_3 Continuous

Optimális megoldást találtunk.
Játék értéke: w* = 0.0
Oszlop kevert stratégiája:
y1 = 0.25
y2 = 0.5
y3 = 0.25
y4 = 0.0


*5. feladat:*
<br>Mátrixjáték modellezése. Van egy kistérség, ahol egy út mentén 4 város található (A-B-C-D), egymástól 10-10 km távolságra. Az A város lakossága 10 ezer fő, B városé 50 ezer fő, C városé 25 ezer fő, D városé pedig 15 ezer fő.
<br> Két áruházlánc (X és Y) szeretne 1-1 áruházat nyitni a kistérségben. Minden lakos abba az áruházba jár, amelyik közelebb van hozzá. Azon lakosok, akiktől egyforma messze vannak, 60% az X, 40% az Y áruházat választja.
<br>a) Írjuk fel a játék kifizetési mátrixát X szemszögéből!

*Megoldás:*
$$
\left[ \begin{array}{cccc}
 60 & 10 & 40 & 60 \\
 90 & 60 & 60 & 75 \\
 70 & 40 & 60 & 85 \\
 40 & 30 & 15 & 60 \\
\end{array} \right]
$$

b) Keressünk *dominált* stratégiákat, amelyeket semmilyen racionális játékos nem választana.

*Megoldás:*
<br>Oszlop második dominálja az elsőt. Oszlop harmadik dominálja a negyediket. Sor második dominálja a harmadikat.

c) Írjuk fel az LP modellt, mely segítségével Nash-egyensúlyi kevert stratégiákat tudunk keresni, és a pulp segítségével oldjuk is meg!

In [8]:
# Megoldás:

from pulp import *

sor = LpProblem('sor', LpMaximize)

x = LpVariable.dicts('x', range(4), lowBound=0, cat='Continuous')
v = LpVariable('v', cat='Continuous')

sor += v

sor += 60*x[0] + 90*x[1] + 70*x[2] + 40*x[3] - v >= 0, "oszlop stratégia 1"
sor += 10*x[0] + 60*x[1] + 40*x[2] + 30*x[3] - v >= 0, "oszlop stratégia 2"
sor += 40*x[0] + 60*x[1] + 60*x[2] + 15*x[3] - v >= 0, "oszlop stratégia 3"
sor += 60*x[0] + 75*x[1] + 85*x[2] + 60*x[3] - v >= 0, "oszlop stratégia 4"
sor += pulp.lpSum([x[i] for i in range(4)]) == 1, "valószínűségi változók"

print(sor)

megoldas = sor.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: v* =', value(sor.objective))
    print('Sor kevert stratégiája: x1 = ', value(x[0]), ', x2 = ', value(x[1]), ', x3 = ', value(x[2]), ', x4 = ', value(x[3]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

sor:
MAXIMIZE
1*v + 0
SUBJECT TO
oszlop_stratégia_1: - v + 60 x_0 + 90 x_1 + 70 x_2 + 40 x_3 >= 0

oszlop_stratégia_2: - v + 10 x_0 + 60 x_1 + 40 x_2 + 30 x_3 >= 0

oszlop_stratégia_3: - v + 40 x_0 + 60 x_1 + 60 x_2 + 15 x_3 >= 0

oszlop_stratégia_4: - v + 60 x_0 + 75 x_1 + 85 x_2 + 60 x_3 >= 0

valószínűségi_változók: x_0 + x_1 + x_2 + x_3 = 1

VARIABLES
v free Continuous
x_0 Continuous
x_1 Continuous
x_2 Continuous
x_3 Continuous

Optimális megoldást találtunk.
Játék értéke: v* = 60.0
Sor kevert stratégiája: x1 =  0.0 , x2 =  1.0 , x3 =  0.0 , x4 =  0.0


In [9]:
#from pulp import *

oszlop = LpProblem('oszlop', LpMinimize)

y = LpVariable.dicts('y', range(4), lowBound=0, cat='Continuous')
w = LpVariable('w', cat='Continuous')

oszlop += w

oszlop += 60*y[0] + 10*y[1] + 40*y[2] + 60*y[3] - w <= 0, "sor stratégia 1"
oszlop += 90*y[0] + 60*y[1] + 60*y[2] + 75*y[3] - w <= 0, "sor stratégia 2"
oszlop += 70*y[0] + 40*y[1] + 60*y[2] + 85*y[3] - w <= 0, "sor stratégia 3"
oszlop += 40*y[0] + 30*y[1] + 15*y[2] + 60*y[3] - w <= 0, "sor stratégia 4"
oszlop += pulp.lpSum([y[i] for i in range(4)]) == 1, "valószínűségi változók"

print(oszlop)

megoldas = oszlop.solve()

if LpStatus[megoldas] == 'Optimal':
    print('Optimális megoldást találtunk.')
    print('Játék értéke: w* = ', value(oszlop.objective))
    print('Oszlop kevert stratégiája: y1 = ', value(y[0]), ', y2 = ', value(y[1]), ', y3 = ', value(y[2]), ', y4 = ', value(y[3]))
elif megoldas == -1:
    print('Az LP-nek nincs megengedett megoldása.')
elif LpStatus[megoldas] == 'Unbounded':
    print('A célfüggvény nem korlátos.')
else:
    print('Az LP megoldása valamely problémába ütközött.')

oszlop:
MINIMIZE
1*w + 0
SUBJECT TO
sor_stratégia_1: - w + 60 y_0 + 10 y_1 + 40 y_2 + 60 y_3 <= 0

sor_stratégia_2: - w + 90 y_0 + 60 y_1 + 60 y_2 + 75 y_3 <= 0

sor_stratégia_3: - w + 70 y_0 + 40 y_1 + 60 y_2 + 85 y_3 <= 0

sor_stratégia_4: - w + 40 y_0 + 30 y_1 + 15 y_2 + 60 y_3 <= 0

valószínűségi_változók: y_0 + y_1 + y_2 + y_3 = 1

VARIABLES
w free Continuous
y_0 Continuous
y_1 Continuous
y_2 Continuous
y_3 Continuous

Optimális megoldást találtunk.
Játék értéke: w* =  60.0
Oszlop kevert stratégiája: y1 =  0.0 , y2 =  1.0 , y3 =  0.0 , y4 =  0.0
